In [1]:
using Pkg
Pkg.activate(".")

using JuMP
using MosekTools
using DynamicPolynomials
using MultivariatePolynomials
using TSSOS

# Parameters 
ϵ = 10^(-5)
d_relax = 2 

@polyvar x0

function interpolation(deg)
    # synthesize Craig interpolation using the standard Putinar theorem
    # deg: degree of interpolation template, h(x,y)
    
    model = Model(optimizer_with_attributes(Mosek.Optimizer))
    set_optimizer_attribute(model, MOI.Silent(), true)
    
    h,hc,hb = add_poly!(model, xvars, deg)
    
    # just avoid bugs when yvars is empty
    if length(yvars)==0
        phivars = xvars
    else
        phivars = [xvars; yvars]
    end
    if length(zvars)==0
        psivars = xvars
    else
        psivars = [xvars; zvars]
    end

    # homogenization
    for i in length(s1)
        model,info1 = add_psatz!(model, h-1,  [phivars; x0], [s1[i]; x0], [1-sum([x0;phivars].^2)], d_relax, QUIET=true, CS=false, TS=false)
    end
    
    for i in length(s2)
        model,info2 = add_psatz!(model, -h-1, [psivars; x0], [s2[i]; x0], [1-sum([x0;psivars].^2)], d_relax, QUIET=true, CS=false, TS=false)
    end
    for i in hc
        @constraint(model, -1<=i<=1)
    end
    # @constraint(model, sum(hc)==1)

    optimize!(model)
    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("termination status: $status")
        status = primal_status(model)
        println("solution status: $status")
    end
    hc = value.(hc)
    for i in 1:length(hc)
        if hc[i] <= ϵ && hc[i] >= -ϵ
            hc[i] = 0
        end
    end
    return hc'*hb
end

function interpolation_homo(deg)
    # synthesize Craig interpolation using the homogenization formulation
    # deg: degree of interpolation template, h(x,y)

    model = Model(optimizer_with_attributes(Mosek.Optimizer))
    set_optimizer_attribute(model, MOI.Silent(), true)
    
    h,hc,hb = add_poly!(model, xvars, deg)
        
    if length(yvars)==0
        phivars = xvars
    else
        phivars = [xvars; yvars]
    end

    if length(zvars)==0
        psivars = xvars
    else
        psivars = [xvars; zvars]
    end

    for i in length(s1)
        model,info = add_psatz!(model, homogenize(h, x0),  [phivars; x0], [homogenize.(s1[i], x0); x0], [1-sum([x0;phivars].^2)], d_relax, QUIET=true, CS=false, TS=true)
    end    
    for i in length(s2)
        model,info = add_psatz!(model, homogenize(-h, x0), [psivars; x0], [homogenize.(s2[i], x0); x0], [1-sum([x0;psivars].^2)], d_relax, QUIET=true, CS=false, TS=true)
    end
    for i in hc
        @constraint(model, -1<=i<=1)
    end
    # @constraint(model, sum(hc)==1)

    optimize!(model)
    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("termination status: $status")
        status = primal_status(model)
        println("solution status: $status")
    end
    hc = value.(hc)      
    for i in 1:length(hc)
        if hc[i] <= ϵ && hc[i] >= -ϵ
            hc[i] = 0
        end
    end
    return hc'*hb
end


  Activating project at `~/Documents/code/Interpolation`


interpolation_homo (generic function with 1 method)

In [3]:
# compute interpolation using two methods

function run_tssos(name, method1, method2)
    include("./Benchmarks/"*name*".jl");

    # print problem instance (so that Mathematica can read)
    file = open("./Results/problem/"*name*".txt", "w");
    write(file, "{")
    for i = 1:length(xvars) 
        if i<=length(xvars)-1
            write(file, string(xvars[i])*",")
        else
            write(file, string(xvars[i]))
        end
    end
    write(file, "}\n")
    write(file, "{")
    for i = 1:length(yvars) 
        if i<=length(yvars)-1
            write(file, string(yvars[i])*",")
        else
            write(file, string(yvars[i]))
        end
    end
    write(file, "}\n")
    write(file, "{")
    for i = 1:length(zvars) 
        if i<=length(zvars)-1
            write(file, string(zvars[i])*",")
        else
            write(file, string(zvars[i]))
        end
    end
    write(file, "}\n")
    for k = [s1, s2]
        write(file, "{")
        for i = 1:length(k)
            write(file, "{")
            for j = 1:length(k[i])
                write(file, Base.replace(string(k[i][j]),"e"=>"*10^"))
                if j < length(k[i])
                    write(file, ",") 
                end
            end
            write(file, "}")
            if i < length(k)
                write(file, ",")
            end
        end
        write(file, "}\n")
    end
    close(file)

    # print results based on the method in CAV20
    file = open("./Results/sufficient/"*name*".txt", "w");
    stats = @timed h = interpolation(deg)
    println("using CAV20 technique:",stats.time)
    @show h
    write(file, Base.replace(string(h),"e"=>"*10^")*"\n")
    write(file, string(stats.time)*"\n") 
    close(file)

    # print homogenization approach results
    file = open("./Results/homo/"*name*".txt", "w");
    stats = @timed h = interpolation_homo(deg)
    println("using homogenization technique:",stats.time)
    @show h
    write(file, Base.replace(string(h),"e"=>"*10^")*"\n")
    write(file, string(stats.time)*"\n") 
    close(file)
end


run_tssos (generic function with 1 method)

In [4]:
# methods
putinar = true
homogenization = true

run_tssos("ex2", putinar, homogenization)

termination status: SLOW_PROGRESS
solution status: UNKNOWN_RESULT_STATUS


using CAV20 technique:12.047120208
h = 

-1.0732375156807408x⁷ - 0.7235586921307784x⁵y² - 0.24022979366802558x³y⁴ - 0.047464621834478145xy⁶ - 0.00042456793651887324x⁶ - 0.0003982702179541268x⁴y² - 0.00019263349544961026x²y⁴ - 4.825027603933034e-5y⁶ - 1.0976233023883983x⁵ - 0.8912015574300153x³y² - 0.413733528317591xy⁴ - 0.0017357272620525917x⁴ - 0.001245959916187371x²y² - 0.00041468526227975y⁴ - 1.1099927366392877x³ - 1.0123693707210413xy² - 0.002951806839474202x² - 0.0024306671945464172y² - 1.1150683546960332x - 0.0007254001105072182


using homogenization technique:0.343331459
h = -0.7450765450825739x⁷ - 0.41167543904896137x⁵y² - 0.3016854672163968x³y⁴ - 0.16065394488305107xy⁶ - 0.849041096115104x⁵ - 0.5108924323024204x³y² - 0.15084823755846208xy⁴ - 0.8348332132397487x³ - 0.6658634215493677xy² - 0.9364035185386194x
